In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [2]:
num_classes = 100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169017344/169001437 [==============================] - 2s 0us/step


In [3]:
x_val = x_train[40000:50000, ]
x_train = x_train[:40000, ]
y_val = y_train[40000:50000]
y_train = y_train[:40000]
print(x_train.shape, y_train.shape)

(40000, 32, 32, 3) (40000, 1)


In [4]:
import cv2
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for keras'
    def __init__(self, images , labels = None,  batch_size = 64,
                 dim = (224, 224), n_classes = 100,
                 shuffle = True):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.dim = dim
        self.n_classes = n_classes
        self.shuffle = shuffle
        
        self.indexes = np.arange(self.images.shape[0]) # (0,1,2,3,...,40000)

        self.on_epoch_end()
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes) 
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.images.shape[0] // self.batch_size 
        
    def __getitem__(self, index):
        'Generate batch of data in position index'
        # index * batch_size : (index +1) *batch_size

        batch_indexes = self.indexes[ index*self.batch_size:(index+1)*self.batch_size]
        
        images =[]
        labels =[] 

        for i in batch_indexes:

          img_i = self.images[i] 
          img_i = cv2.resize(img_i , self.dim )
          img_i = (img_i) / 127. - 1 

          # augm.... 
          images.append(img_i)

          label_i = self.labels[i]
          labels.append(label_i)
      
        images = np.stack(images) # batch_size, 224,224,3
        labels = np_utils.to_categorical(labels, self.n_classes)

        return images, labels 

In [5]:
resnet_model = ResNet50(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))
avg = GlobalAveragePooling2D()(resnet_model.output)

den1 = Dense(512, activation='relu')(avg)

drop1 = Dropout(0.5)(den1)
den2 = Dense(num_classes, activation='softmax')(drop1)

model = tf.keras.Model(inputs = resnet_model.inputs, outputs =[den2])


94781440/94765736 [==============================] - 1s 0us/step


In [6]:
for layer in resnet_model.layers:
  if isinstance(layer, BatchNormalization):
    layer.trainable = True
  else:
    layer.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
data_gen = DataGenerator(
    x_train,
    y_train,
    batch_size = 64,
    dim = (224, 224), n_classes = 100,
    shuffle = True
)
val_gen  = DataGenerator(
    x_val,
    y_val,
    batch_size = 64,
    dim = (224, 224), n_classes = 100,
    shuffle = False
)

Freeze pre-train and warm up top layers

In [8]:
t=time.time()
bt_size = 64 
Epoch = 5
historytemp = model.fit(data_gen,
                        validation_data=val_gen,
                        epochs=Epoch)
print('Training time: %s' % (time.time() - t))

Epoch 1/5
625/625 [==============================] - 704s 1s/step - loss: 2.4164 - accuracy: 0.3837 - val_loss: 4.7319 - val_accuracy: 0.1036
Epoch 2/5
625/625 [==============================] - 665s 1s/step - loss: 1.3875 - accuracy: 0.6037 - val_loss: 1.0491 - val_accuracy: 0.6902
Epoch 3/5
625/625 [==============================] - 665s 1s/step - loss: 1.1292 - accuracy: 0.6695 - val_loss: 0.9443 - val_accuracy: 0.7171
Epoch 4/5
625/625 [==============================] - 666s 1s/step - loss: 0.9978 - accuracy: 0.7015 - val_loss: 0.8995 - val_accuracy: 0.7327
Epoch 5/5
625/625 [==============================] - 663s 1s/step - loss: 0.8865 - accuracy: 0.7311 - val_loss: 0.8631 - val_accuracy: 0.7434
Training time: 3381.4866099357605


In [9]:
test_gen = DataGenerator(
    x_test,
    y_test,
    batch_size = 64,
    dim = (224, 224), n_classes = 100,
    shuffle = False
)


In [10]:
model.evaluate(test_gen)

156/156 [==============================] - 65s 417ms/step - loss: 0.8384 - accuracy: 0.7554


[0.8384006023406982, 0.7554086446762085]

Unfreeze pre-train model, update entire model

In [11]:
# unfreeze weights
for layer in resnet_model.layers:
    layer.trainable = True

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min',
                   patience = 8, restore_best_weights = True, verbose = 1)

In [14]:
hist = model.fit(data_gen,
                        validation_data=val_gen,
                  epochs=25,
                  callbacks=[es],
                  verbose=1
                  )


Epoch 1/25
625/625 [==============================] - 880s 1s/step - loss: 0.7514 - accuracy: 0.7679 - val_loss: 0.8380 - val_accuracy: 0.7562
Epoch 2/25
625/625 [==============================] - 872s 1s/step - loss: 0.6663 - accuracy: 0.7954 - val_loss: 0.7996 - val_accuracy: 0.7644
Epoch 3/25
625/625 [==============================] - 872s 1s/step - loss: 0.6095 - accuracy: 0.8087 - val_loss: 0.7833 - val_accuracy: 0.7701
Epoch 4/25
625/625 [==============================] - 872s 1s/step - loss: 0.5619 - accuracy: 0.8236 - val_loss: 0.8090 - val_accuracy: 0.7619
Epoch 5/25
625/625 [==============================] - 872s 1s/step - loss: 0.5262 - accuracy: 0.8331 - val_loss: 0.7752 - val_accuracy: 0.7751
Epoch 6/25
625/625 [==============================] - 872s 1s/step - loss: 0.4882 - accuracy: 0.8463 - val_loss: 0.7625 - val_accuracy: 0.7741
Epoch 7/25
625/625 [==============================] - 874s 1s/step - loss: 0.4525 - accuracy: 0.8547 - val_loss: 0.7593 - val_accuracy: 0.7795

In [15]:
model.save_weights("./model_dense512_drop05.h5")

In [16]:
model.evaluate(test_gen)

156/156 [==============================] - 65s 416ms/step - loss: 0.7346 - accuracy: 0.7902


[0.734562873840332, 0.7901642918586731]